# Test ONNX Model with Label Matching
This notebook tests the ONNX model by matching a sample text to predefined labels using cosine similarity.

In [23]:
# Import required libraries
from transformers import AutoTokenizer
from onnxruntime import InferenceSession
import numpy as np

# Define paths to the model and tokenizer
onnx_model_path = '../artifacts/model/model.onnx'
tokenizer_dir = '../artifacts/model'

# Define labels and text
labels = ['Shopping', 'Exercise', 'Hobbies']
text = 'I need to buy chicken and rolls for dinner.'

# Load ONNX model and tokenizer
session = InferenceSession(onnx_model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

# Helper function to tokenize text
def tokenize(to_tokenize):
    tokens = tokenizer(to_tokenize, return_tensors='np', padding=True, truncation=True)
    return {k: v for k, v in tokens.items() if k in [input.name for input in session.get_inputs()]}

# Tokenize text and get embeddings
text_tokens = tokenize(text)
text_embedding = session.run(None, text_tokens)[0].mean(axis=1)

# Tokenize labels and get embeddings
label_tokens = [tokenize(label) for label in labels]
label_embeddings = [session.run(None, token) for token in label_tokens]

# Compute cosine similarity between text and labels
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarities = [cosine_similarity(text_embedding[0], label[0].mean(axis=1)[0]) for label in label_embeddings]

# Find the best matching label
best_label = labels[np.argmax(similarities)]
print(f'Sample text: "{text}"')
print(f'Matched label: {best_label}')

Sample text: "I need to buy chicken and rolls for dinner."
Matched label: Shopping
